# 動物画像分類システム - CNN実装プロジェクト

## プロジェクト内容

畳み込みニューラルネットワーク（CNN）を使用した猫と犬の画像分類システムです。TensorFlow Datasetsから取得した高品質な動物画像データを用いて、深層学習による画像認識モデルを構築しました。TensorFlowによるCNN実装とコンピュータビジョン技術を学習することを目的として開発しました。


In [1]:
import warnings
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# 警告メッセージを非表示にする
warnings.filterwarnings('ignore')

# プロジェクトのベースディレクトリを設定
base_dir = os.path.dirname(os.getcwd())
dataset_dir = os.path.join(base_dir, 'image_datasets')

# データセット用フォルダ構造を作成
folders = [
    'training_images/cats',
    'training_images/dogs', 
    'test_images/cats',
    'test_images/dogs',
    'prediction_samples'
]

for folder in folders:
    folder_path = os.path.join(dataset_dir, folder)
    os.makedirs(folder_path, exist_ok=True)
    print(f"フォルダを作成しました: {folder_path}")

# TensorFlow Datasetsから猫と犬のデータセットをダウンロード
print("猫と犬の画像データセットをダウンロード中...")
dataset, info = tfds.load(
    'cats_vs_dogs',
    with_info=True,
    as_supervised=True,
    data_dir=os.path.join(base_dir, 'tensorflow_datasets')
)

# データセット情報を表示
print(f"データセット名: {info.name}")
print(f"総画像数: {info.splits['train'].num_examples}")
print(f"クラス数: {info.features['label'].num_classes}")
print(f"クラス名: {info.features['label'].names}")

# データセットを取得
train_dataset = dataset['train']

def save_images_from_dataset(dataset, save_path, class_name, target_class, max_count):
    """データセットから指定されたクラスの画像を保存する関数"""
    count = 0
    for image, label in dataset:
        if count >= max_count:
            break
        
        if label.numpy() == target_class:
            # 画像を0-255の範囲に変換
            image_array = tf.cast(image, tf.uint8).numpy()
            
            # PIL画像として保存
            pil_image = Image.fromarray(image_array)
            
            # ファイル名を生成
            filename = f"{class_name}_{count+1:04d}.jpg"
            filepath = os.path.join(save_path, filename)
            
            # 画像を保存
            pil_image.save(filepath)
            count += 1
    
    print(f"{class_name}の画像を{count}枚保存しました")
    return count

# データセットをリストに変換（効率的な処理のため）
dataset_list = list(train_dataset.take(25000))  # 全データセットを取得
np.random.shuffle(dataset_list)  # シャッフル

# 猫の画像を分離（ラベル0が猫）
cat_images = [(img, lbl) for img, lbl in dataset_list if lbl.numpy() == 0]
# 犬の画像を分離（ラベル1が犬）
dog_images = [(img, lbl) for img, lbl in dataset_list if lbl.numpy() == 1]

print(f"利用可能な猫の画像: {len(cat_images)}枚")
print(f"利用可能な犬の画像: {len(dog_images)}枚")

# 訓練用猫画像を保存（8000枚）
train_cats_path = os.path.join(dataset_dir, 'training_images', 'cats')
saved_count = 0
for i, (image, label) in enumerate(cat_images[:8000]):
    image_array = tf.cast(image, tf.uint8).numpy()
    pil_image = Image.fromarray(image_array)
    filename = f"cat_{saved_count+1:04d}.jpg"
    filepath = os.path.join(train_cats_path, filename)
    pil_image.save(filepath)
    saved_count += 1

print(f"訓練用猫画像を{saved_count}枚保存しました")

# 訓練用犬画像を保存（8000枚）
train_dogs_path = os.path.join(dataset_dir, 'training_images', 'dogs')
saved_count = 0
for i, (image, label) in enumerate(dog_images[:8000]):
    image_array = tf.cast(image, tf.uint8).numpy()
    pil_image = Image.fromarray(image_array)
    filename = f"dog_{saved_count+1:04d}.jpg"
    filepath = os.path.join(train_dogs_path, filename)
    pil_image.save(filepath)
    saved_count += 1

print(f"訓練用犬画像を{saved_count}枚保存しました")

# テスト用猫画像を保存（2000枚）
test_cats_path = os.path.join(dataset_dir, 'test_images', 'cats')
saved_count = 0
for i, (image, label) in enumerate(cat_images[8000:10000]):
    image_array = tf.cast(image, tf.uint8).numpy()
    pil_image = Image.fromarray(image_array)
    filename = f"cat_{saved_count+1:04d}.jpg"
    filepath = os.path.join(test_cats_path, filename)
    pil_image.save(filepath)
    saved_count += 1

print(f"テスト用猫画像を{saved_count}枚保存しました")

# テスト用犬画像を保存（2000枚）
test_dogs_path = os.path.join(dataset_dir, 'test_images', 'dogs')
saved_count = 0
for i, (image, label) in enumerate(dog_images[8000:10000]):
    image_array = tf.cast(image, tf.uint8).numpy()
    pil_image = Image.fromarray(image_array)
    filename = f"dog_{saved_count+1:04d}.jpg"
    filepath = os.path.join(test_dogs_path, filename)
    pil_image.save(filepath)
    saved_count += 1

print(f"テスト用犬画像を{saved_count}枚保存しました")

# 予測用サンプル画像を作成
prediction_samples_dir = os.path.join(dataset_dir, 'prediction_samples')

# 猫のサンプル画像
if len(cat_images) > 10000:
    sample_cat_image, _ = cat_images[10000]
    cat_array = tf.cast(sample_cat_image, tf.uint8).numpy()
    cat_sample_path = os.path.join(prediction_samples_dir, 'sample_cat.jpg')
    Image.fromarray(cat_array).save(cat_sample_path)
    print(f"猫のサンプル画像を保存しました: {cat_sample_path}")

# 犬のサンプル画像
if len(dog_images) > 10000:
    sample_dog_image, _ = dog_images[10000]
    dog_array = tf.cast(sample_dog_image, tf.uint8).numpy()
    dog_sample_path = os.path.join(prediction_samples_dir, 'sample_dog.jpg')
    Image.fromarray(dog_array).save(dog_sample_path)
    print(f"犬のサンプル画像を保存しました: {dog_sample_path}")

print("\nデータセットの準備が完了しました！")
print(f"訓練用画像: 猫8000枚、犬8000枚")
print(f"テスト用画像: 猫2000枚、犬2000枚")
print(f"予測用サンプル: 各クラス1枚ずつ")

2025-06-20 06:37:24.886482: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


フォルダを作成しました: /Users/y_suzuki/Documents/PythonPortfolioProject/DeepLearning/animal_image_classifier/image_datasets/training_images/cats
フォルダを作成しました: /Users/y_suzuki/Documents/PythonPortfolioProject/DeepLearning/animal_image_classifier/image_datasets/training_images/dogs
フォルダを作成しました: /Users/y_suzuki/Documents/PythonPortfolioProject/DeepLearning/animal_image_classifier/image_datasets/test_images/cats
フォルダを作成しました: /Users/y_suzuki/Documents/PythonPortfolioProject/DeepLearning/animal_image_classifier/image_datasets/test_images/dogs
フォルダを作成しました: /Users/y_suzuki/Documents/PythonPortfolioProject/DeepLearning/animal_image_classifier/image_datasets/prediction_samples
猫と犬の画像データセットをダウンロード中...
データセット名: cats_vs_dogs
総画像数: 23262
クラス数: 2
クラス名: ['cat', 'dog']


2025-06-20 06:37:56.484109: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


利用可能な猫の画像: 11658枚
利用可能な犬の画像: 11604枚
訓練用猫画像を8000枚保存しました
訓練用犬画像を8000枚保存しました
テスト用猫画像を2000枚保存しました
テスト用犬画像を2000枚保存しました
猫のサンプル画像を保存しました: /Users/y_suzuki/Documents/PythonPortfolioProject/DeepLearning/animal_image_classifier/image_datasets/prediction_samples/sample_cat.jpg
犬のサンプル画像を保存しました: /Users/y_suzuki/Documents/PythonPortfolioProject/DeepLearning/animal_image_classifier/image_datasets/prediction_samples/sample_dog.jpg

データセットの準備が完了しました！
訓練用画像: 猫8000枚、犬8000枚
テスト用画像: 猫2000枚、犬2000枚
予測用サンプル: 各クラス1枚ずつ


In [2]:
# ディープラーニングフレームワーク
import tensorflow as tf
# 画像データ前処理のためのライブラリ
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# TensorFlowのバージョンを確認
tf.__version__

'2.16.2'

In [4]:
# 訓練用画像データの前処理設定を作成
train_datagen = ImageDataGenerator(
    rescale=1./255,           # ピクセル値を0-1の範囲に正規化
    shear_range=0.2,          # シアー変換（画像の歪み）を最大20%適用
    zoom_range=0.2,           # ズーム変換を最大20%適用  
    horizontal_flip=True      # 水平反転をランダムに適用
)

In [5]:
# 訓練用画像データを読み込み、前処理を適用
training_set = train_datagen.flow_from_directory(
    '../image_datasets/training_images',  # 訓練画像が格納されているディレクトリ
    target_size=(64, 64),                 # 画像サイズを64x64ピクセルにリサイズ
    batch_size=32,                        # 1回に処理する画像数を32枚に設定
    class_mode='binary'                   # 二値分類（猫と犬）を指定
)

Found 16000 images belonging to 2 classes.


In [6]:
# テスト用画像データの前処理設定を作成（データ拡張なし）
test_datagen = ImageDataGenerator(rescale=1./255)  # 正規化のみ適用

In [7]:
# テスト用画像データを読み込み、前処理を適用
test_set = test_datagen.flow_from_directory(
    '../image_datasets/test_images',      # テスト画像が格納されているディレクトリ
    target_size=(64, 64),                 # 画像サイズを64x64ピクセルにリサイズ
    batch_size=32,                        # 1回に処理する画像数を32枚に設定
    class_mode='binary'                   # 二値分類（猫と犬）を指定
)

Found 4000 images belonging to 2 classes.


In [8]:
# Sequentialモデルで畳み込みニューラルネットワークを初期化
cnn = tf.keras.models.Sequential()

In [9]:
# 第1畳み込み層を追加
cnn.add(tf.keras.layers.Conv2D(
    filters=32,                    # フィルター（特徴検出器）の数を32個に設定
    kernel_size=3,                 # カーネル（フィルター）のサイズを3x3に設定
    activation='relu',             # 活性化関数にReLUを使用
    input_shape=[64, 64, 3]       # 入力画像の形状（64x64ピクセル、3チャンネル）
))

In [10]:
# 第1最大プーリング層を追加
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,      # プーリング窓のサイズを2x2に設定
    strides=2         # プーリング窓の移動幅を2ピクセルに設定
))

In [11]:
# 第2畳み込み層を追加
cnn.add(tf.keras.layers.Conv2D(
    filters=32,            # フィルター数を32個に設定
    kernel_size=3,         # カーネルサイズを3x3に設定
    activation='relu'      # 活性化関数にReLUを使用
))

In [12]:
# 第2最大プーリング層を追加
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,      # プーリング窓のサイズを2x2に設定
    strides=2         # プーリング窓の移動幅を2ピクセルに設定
))

In [13]:
# フラット化層を追加（2次元の特徴マップを1次元ベクトルに変換）
cnn.add(tf.keras.layers.Flatten())

In [14]:
# 全結合隠れ層を追加
cnn.add(tf.keras.layers.Dense(
    units=128,             # ニューロン数を128個に設定
    activation='relu'      # 活性化関数にReLUを使用
))

In [15]:
# 出力層を追加
cnn.add(tf.keras.layers.Dense(
    units=1,               # 出力ニューロン数を1個に設定（二値分類のため）
    activation='sigmoid'   # 活性化関数にSigmoidを使用
))

In [16]:
# CNNをコンパイル（学習の設定を行う）
cnn.compile(
    optimizer='adam',              # 最適化アルゴリズムにAdamを使用
    loss='binary_crossentropy',   # 損失関数に二値交差エントロピーを使用
    metrics=['accuracy']           # 評価指標に精度を使用
)

In [17]:
# 訓練データでモデルを学習し、テストデータで同時評価
cnn.fit(
    x=training_set,              # 学習に使用する訓練データ
    validation_data=test_set,    # 各エポック終了時に評価を行うテストデータ
    epochs=25                    # 学習回数（エポック数）を25回に設定
)

Epoch 1/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 113s 222ms/step - accuracy: 0.6193 - loss: 0.6450 - val_accuracy: 0.7385 - val_loss: 0.5200
Epoch 2/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 113s 226ms/step - accuracy: 0.7270 - loss: 0.5370 - val_accuracy: 0.7675 - val_loss: 0.4816
Epoch 3/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 106s 211ms/step - accuracy: 0.7604 - loss: 0.4917 - val_accuracy: 0.7800 - val_loss: 0.4612
Epoch 4/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 96s 191ms/step - accuracy: 0.7765 - loss: 0.4590 - val_accuracy: 0.7868 - val_loss: 0.4478
Epoch 5/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 101s 202ms/step - accuracy: 0.7932 - loss: 0.4409 - val_accuracy: 0.7825 - val_loss: 0.4496
Epoch 6/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 97s 194ms/step - accuracy: 0.8035 - loss: 0.4215 - val_accuracy: 0.8008 - val_loss: 0.4290
Epoch 7/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 98s 195ms/step - accuracy: 0.8105 - loss: 0.4152 - val_accuracy: 0.8120 - val_loss: 0.4146
Epoch 8/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 101s 202ms/step - accuracy: 0.8237 - lo

In [18]:
# 数値計算用ライブラリ
import numpy as np
# 画像処理用ライブラリ
from tensorflow.keras.preprocessing import image

In [19]:
# 予測用画像を読み込み（猫のサンプル画像を使用）
test_image = image.load_img(
    '../image_datasets/prediction_samples/sample_cat.jpg',  # 予測対象の画像ファイルパス
    target_size=(64, 64)                                    # 画像サイズを64x64に調整
)

In [20]:
# PIL画像をNumPy配列に変換
test_image = image.img_to_array(test_image)

In [21]:
# バッチ次元を追加（モデルはバッチ入力を期待するため）
test_image = np.expand_dims(test_image, axis=0)

In [22]:
# CNNモデルで予測を実行
result = cnn.predict(test_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step


In [23]:
# クラスインデックスを確認（どちらが猫でどちらが犬かを確認）
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [24]:
# 予測結果を二値分類で判定
if result[0][0] == 1:
    prediction = 'dog'      # 結果が1なら犬と判定
else:
    prediction = 'cat'      # 結果が0なら猫と判定

# 予測結果を表示
print(prediction)

cat
